In [1]:
from pprint import pprint
import itertools
import re

input_file = "inputs/aoc_input_19.txt"
desired_rule = 0

In [2]:
if True:
    with open(input_file) as f:
        content = f.read()
else:
    content = '''
42: 9 14 | 10 1
9: 14 27 | 1 26
10: 23 14 | 28 1
1: "a"
11: 42 31
5: 1 14 | 15 1
19: 14 1 | 14 14
12: 24 14 | 19 1
16: 15 1 | 14 14
31: 14 17 | 1 13
6: 14 14 | 1 14
2: 1 24 | 14 4
0: 8 11
13: 14 3 | 1 12
15: 1 | 14
17: 14 2 | 1 7
23: 25 1 | 22 14
28: 16 1
4: 1 1
20: 14 14 | 1 15
3: 5 14 | 16 1
27: 1 6 | 14 18
14: "b"
21: 14 1 | 1 14
25: 1 1 | 1 14
22: 14 14
8: 42
26: 14 22 | 1 20
18: 15 15
7: 14 5 | 1 21
24: 14 1

abbbbbabbbaaaababbaabbbbabababbbabbbbbbabaaaa
bbabbbbaabaabba
babbbbaabbbbbabbbbbbaabaaabaaa
aaabbbbbbaaaabaababaabababbabaaabbababababaaa
bbbbbbbaaaabbbbaaabbabaaa
bbbababbbbaaaaaaaabbababaaababaabab
ababaaaaaabaaab
ababaaaaabbbaba
baabbaaaabbaaaababbaababb
abbbbabbbbaaaababbbbbbaaaababb
aaaaabbaabaaaaababaa
aaaabbaaaabbaaa
aaaabbaabbaaaaaaabbbabbbaaabbaabaaa
babaaabbbaaabaababbaabababaaab
aabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba
'''
content = content.strip()
rules, messages = [x.strip().split('\n') for x in content.split('\n\n')]

In [3]:
def int_or_none(n):
    try:
        return int(n)
    except:
        return None
    

def generate_rules(rules):
    rule_map = {}
    referenced_rules = {}
    for rule in rules:
        rule_number, rule_text = rule.split(':')
        rule_number = int(rule_number)

        rule_map[rule_number] = rule_text.strip().split()

        rule_text_split = [int(x) for x in rule_text.split() if int_or_none(x)]
        referenced_rules[rule_number] = set(rule_text_split)

    rule_processing_order = []
    while len(rule_processing_order) != len(rules):
        zero_reference_rules = [[key, value] for key, value in referenced_rules.items() if len(value) == 0 and key not in rule_processing_order]
        zero_reference_rules_indexes = [x for x,_ in zero_reference_rules]
        rule_processing_order = rule_processing_order + zero_reference_rules_indexes

        for x in referenced_rules:
            referenced_rules[x] = referenced_rules[x] - set(zero_reference_rules_indexes)

    rule_index_set = set([str(x) for x in rule_processing_order])

    for rule_index in rule_processing_order:
        rule_content = rule_map[rule_index]
        for i in range(len(rule_content)):
            if rule_content[i] in rule_index_set:
                if len(rule_map[int(rule_content[i])]) == 1:
                    rule_content[i] = rule_map[int(rule_content[i])]
                else:
                    rule_content[i] = '(' + rule_map[int(rule_content[i])] + ')'
            rule_content[i] = rule_content[i].replace('"', '')

        rule_map[rule_index] = ''.join(rule_content)
        
    return rule_map

In [4]:
rule_map = generate_rules(rules)
final_rule = '^(' + rule_map[desired_rule] + ')$'

len([message for message in messages if len(re.findall(final_rule, message))])

124

In [5]:
new_rules = rules[:]

new_rules[[x for x,y in enumerate(rules) if y.startswith('8:')][0]] = '8: 42 +'

# Given that regex is a state machine, it is unable to calculate `11: 42 31 | 42 11 31` for an infinite number of recursions.
# So I just hardcoded an X amount, and stopped increasing it whenever the numbers stopped changing
max_recurs = 5
new_rules[[x for x,y in enumerate(rules) if y.startswith('11:')][0]] = '11: ' + ' | '.join([' '.join(['42']*n + ['31']*n) for n in range(1, max_recurs+1)])

rule_map = generate_rules(new_rules)
final_rule = '^(' + rule_map[desired_rule] + ')$'

len([message for message in messages if len(re.findall(final_rule, message))])

228